In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OrdinalEncoder
import re

In [7]:
df = pd.read_csv('Cars\cars.csv')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\migue\AppData\Local\Temp\ipykernel_14016\2405752358.py:1: SyntaxWarning: invalid escape sequence '\c'
  df = pd.read_csv('Cars\cars.csv')


In [8]:
def split_features(text):
    if pd.isna(text):
        return []
    return [feature.strip() for feature in text.split(',')]

In [9]:
# Aplicar la función a las columnas relevantes
columns_to_encode = ['Comfort', 'Entertainment', 'Extras', 'Security']
for col in columns_to_encode:
    df[col] = df[col].apply(split_features)

mlb = MultiLabelBinarizer()

# Realizar el one-hot encoding para cada columna
for col in columns_to_encode:
    encoded = pd.DataFrame(mlb.fit_transform(df[col]),
                           columns=mlb.classes_,
                           index=df.index)
    
    
    encoded = encoded.add_prefix(f'{col}_')
    
    # Concatenar las nuevas columnas al DataFrame original
    df = pd.concat([df, encoded], axis=1)

# Eliminar las columnas originales si es necesario
df = df.drop(columns=columns_to_encode)

In [10]:
df['Color_interior'] = df['Color_interior'].fillna('Black')
df['Color_exterior'] = df['Color_exterior'].replace('SPARKLING BROWN METALLIC', 'Black')
df['Owners'] = df['Owners'].fillna(1)

In [ ]:
from deep_translator import GoogleTranslator
# Crear un traductor de alemán a inglés
def translate_column_names(df):
    translator = GoogleTranslator(source='de', target='en')
    
    translations = {}
    
    for column in df.columns:
        try:
            translated = translator.translate(column)

            translations[column] = translated
        except Exception as e:
            print(f"Error al traducir '{column}': {e}")

            translations[column] = column
    
    # Renombrar las columnas del DataFrame
    df.rename(columns=translations, inplace=True)
    
    return df

translate_column_names(df)

In [12]:
def translate_value(value):
        translator = GoogleTranslator(source='de', target='en')
        if pd.isna(value):  # Manejar valores NaN
            return value
        try:
            # Convertir a string si no lo es
            value = str(value)
            translated = translator.translate(value)
            return translated
        except Exception as e:
            print(f"Error al traducir '{value}': {e}")
            return value
        
columns = ['Color_exterior','Color_interior','Fuel','Gearbox','Seller']
for column in columns:
    print(f"Traduciendo columna: {column}")
    df[column] = df[column].apply(translate_value)

df.head()

Traduciendo columna: Color_exterior
Traduciendo columna: Color_interior
Traduciendo columna: Fuel
Traduciendo columna: Gearbox
Traduciendo columna: Seller


,Color_exterior,Color_interior,Fuel,Gearbox,Initial_registration,Mileage,Owners,power,Seller,price,...,Security_Lane Keeping Assist,Security_Daytime running lights,Security_Blind Spot Assistant,Security_Traction Control,Security_Traffic Sign Recognition,Security_Full-LED headlights,Security_Immobilizer,Security_Xenon headlights,Security_Central locking,Security_Central locking with radio remote control
0,Gray,Black,petrol,Automatic,10/2023,5185,1.0,100 kW (136 PS),Dealer,31489,...,1,1,0,1,1,1,1,0,1,1
1,Blue,Black,petrol,Manual transmission,07/2020,44550,1.0,103 kW (140 PS),Dealer,20490,...,1,1,0,1,1,0,1,0,1,0
2,Gray,Black,Diesel,Automatic,04/2016,104115,2.0,110 kW (150 PS),Dealer,18750,...,0,1,0,1,1,0,1,0,1,0
3,Black,Other,petrol,Manual transmission,01/2021,22943,1.0,100 kW (136 PS),Dealer,22890,...,1,1,0,1,1,0,1,0,1,0
4,Gray,Black,petrol,Automatic,12/2022,37500,1.0,100 kW (136 PS),Dealer,28360,...,1,1,1,1,1,0,1,0,1,0


In [13]:
df['Owners'] = df['Owners'].astype('int64')
columnas_object = df.select_dtypes(include=['object']).columns
columnas_object = [col for col in columnas_object if col != 'power']

In [14]:
oe = OrdinalEncoder(dtype=np.int64)

df[columnas_object] = oe.fit_transform(df[columnas_object])

In [15]:
def extraer_ps(valor):
    match = re.search(r'(\d+)\s*PS', valor)
    if match:
        return int(match.group(1))
    else:
        return None
    
df['power'] = df['power'].apply(extraer_ps)

In [16]:
df.info()#Todas las columnas en type int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Columns: 136 entries, Color_exterior to Security_Central locking with radio remote control
dtypes: int64(136)
memory usage: 374.1 KB


In [17]:
df.to_csv('cars_Cleaned.csv',index = False)